In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content
!rm -r jawiOCR
!git clone https://github.com/mirzasyhm/jawiOCR.git
%cd /content/jawiOCR/parseq_jawi/

/content
Cloning into 'jawiOCR'...
remote: Enumerating objects: 688, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 688 (delta 30), reused 46 (delta 20), pack-reused 627 (from 1)
Receiving objects: 100% (688/688), 102.81 MiB | 32.43 MiB/s, done.
Resolving deltas: 100% (299/299), done.
/content/jawiOCR/parseq_jawi


In [3]:
# Step 1: Delete existing data directory if exists
!rm -rf data/

# Step 2: Download the LMDB tar file
!wget https://huggingface.co/datasets/mirzasyhm/synthetic_jawi_images/resolve/main/data_words_split_lmdb.tar -O jawi.tar

# Step 3: Extract the tar file
!tar -xf jawi.tar

# Step 4: Rename the extracted folder to "data"
!mv data_words_split_lmdb data/


--2025-06-08 14:46:59--  https://huggingface.co/datasets/mirzasyhm/synthetic_jawi_images/resolve/main/data_words_split_lmdb.tar
Resolving huggingface.co (huggingface.co)... 65.8.243.92, 65.8.243.16, 65.8.243.90, ...
Connecting to huggingface.co (huggingface.co)|65.8.243.92|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/f9/16/f916668cb9026f1a501d38e183f8b89b65d2f31a4159ab1330ade73ab5a5fc1e/b7be426d1d3c4ad6dff548bb58960b963870c7eedea7c9c79caecb23818fa659?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27data_words_split_lmdb.tar%3B+filename%3D%22data_words_split_lmdb.tar%22%3B&response-content-type=application%2Fx-tar&Expires=1749397619&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0OTM5NzYxOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2Y5LzE2L2Y5MTY2NjhjYjkwMjZmMWE1MDFkMzhlMTgzZjhiODliNjVkMmYzMWE0MTU5YWIxMzMwYWRlNzNhYjVhNWZjMWUvYjdiZTQyNmQxZDNjNGFkN

In [4]:
!mkdir -p data/train/jawi data/test/jawi data/val/jawi && \
mv data/train/*.mdb data/train/jawi/ && \
mv data/test/*.mdb data/test/jawi/ && \
mv data/val/*.mdb data/val/jawi/


In [5]:
!pip install pip-tools
!make clean-reqs reqs  # Regenerate all the requirements files

rm -f requirements/constraints.txt requirements/core.*.txt requirements/bench.txt requirements/core.txt requirements/dev.txt requirements/test.txt requirements/train.txt requirements/tune.txt
CONSTRAINTS=/dev/null pip-compile --quiet --no-header --allow-unsafe --resolver=backtracking --strip-extras --output-file requirements/constraints.txt requirements/bench.in requirements/core.in requirements/dev.in requirements/test.in requirements/train.in requirements/tune.in --extra-index-url https://download.pytorch.org/whl/cpu
    Preparing metadata (setup.py) ... done
    Preparing metadata (setup.py) ... done
    Preparing metadata (setup.py) ... done
CONSTRAINTS=constraints.txt pip-compile --quiet --no-header --allow-unsafe --resolver=backtracking --strip-extras --no-annotate --output-file requirements/bench.txt requirements/bench.in
    Preparing metadata (setup.py) ... done
    Preparing metadata (setup.py) ... done
    Preparing metadata (setup.py) ... done
CONSTRAINTS=constraints.txt pi

In [6]:
platform = "cu121"

!make torch-{platform}
!pip install -r requirements/core.{platform}.txt -e .[train,test]

Generating requirements/core.cu121.txt
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
Obtaining file:///content/jawiOCR/parseq_jawi
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for strhub (pyproject.toml) ... done
  Created wheel for strhub: filename=strhub-1.2.0-0.editable-py3-none-any.whl size=17049 sha256=2bdaea78b855a907ac5e23d70fc8fec0774d217c4862524849429d3c9cbd932a
  Stored in directory: /tmp/pip-ephem-wheel-cache-_76i60dp/wheels/b8/fc/79/177eca182221e0f95b57302acb3216e4e056e2bf919c89a610
Successfully built strhub
  Attempting uninstall: strhub
    Found existing installation: strhub 1.2.0
    Uninstalling strhub-1.2.0:
      Successfully uninstalled strhub-1.2.0


In [7]:
import torch, os, platform
print("CUDA visible :", torch.cuda.is_available())
print("Device       :", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "-")
print("Torch/cuDNN   ", torch.version.cuda, torch.backends.cudnn.version())


CUDA visible : True
Device       : NVIDIA A100-SXM4-40GB
Torch/cuDNN    12.1 90100


In [8]:
import torch, numpy, tensorboard
import google.protobuf as protobuf
print("Torch  :", torch.__version__, torch.version.cuda)
print("NumPy  :", numpy.__version__)
print("Proto  :", protobuf.__version__)
print("CUDA OK:", torch.cuda.is_available(), torch.cuda.get_device_name(0))

Torch  : 2.5.1+cu121 12.1
NumPy  : 1.26.4
Proto  : 5.29.4
CUDA OK: True NVIDIA A100-SXM4-40GB


In [ ]:
%cd /content/parseq_jawi/
!python train.py

/content/parseq_jawi
/content/parseq_jawi/train.py:64: DeprecationWarning: torch.get_autocast_gpu_dtype() is deprecated. Please use torch.get_autocast_dtype('cuda') instead. (Triggered internally at ../torch/csrc/autograd/init.cpp:787.)
  config.trainer.precision = 'bf16-mixed' if torch.get_autocast_gpu_dtype() is torch.bfloat16 else '16-mixed'
/usr/local/lib/python3.11/dist-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
  | Name             | Type           | Params | Mode 
------------------------------------------------------------
0 | model            | PARSeq         | 23.9 M | train
1 | model.encoder    | Encoder        | 21.4 M | train
2 | model.decoder    | Decoder        | 2.4 M  | train
3 | model.head       | Linear         | 50.1 K | train
4 | model.text_embed | TokenEmbedding | 50.7 K | t

In [ ]:
!python train_with_metric.py

/content/parseq_jawi/train_with_metric.py:86: DeprecationWarning: torch.get_autocast_gpu_dtype() is deprecated. Please use torch.get_autocast_dtype('cuda') instead. (Triggered internally at ../torch/csrc/autograd/init.cpp:787.)
  config.trainer.precision = 'bf16-mixed' if torch.get_autocast_gpu_dtype() is torch.bfloat16 else '16-mixed'
/usr/local/lib/python3.11/dist-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
  | Name             | Type           | Params | Mode 
------------------------------------------------------------
0 | model            | PARSeq         | 23.9 M | train
1 | model.encoder    | Encoder        | 21.4 M | train
2 | model.decoder    | Decoder        | 2.4 M  | train
3 | model.head       | Linear         | 50.1 K | train
4 | model.text_embed | TokenEmbedding | 50.7 K | train
5 | 

In [25]:
!python train_with_metric.py +experiment=parseq ckpt_path=outputs/parseq/2025-05-12_10-45-42/checkpoints/last.ckpt

/content/jawiOCR/parseq_jawi/train_with_metric.py:86: DeprecationWarning: torch.get_autocast_gpu_dtype() is deprecated. Please use torch.get_autocast_dtype('cuda') instead. (Triggered internally at ../torch/csrc/autograd/init.cpp:787.)
  config.trainer.precision = 'bf16-mixed' if torch.get_autocast_gpu_dtype() is torch.bfloat16 else '16-mixed'
/usr/local/lib/python3.11/dist-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
  | Name             | Type           | Params | Mode 
------------------------------------------------------------
0 | model            | PARSeq         | 23.9 M | train
1 | model.encoder    | Encoder        | 21.4 M | train
2 | model.decoder    | Decoder        | 2.4 M  | train
3 | model.head       | Linear         | 50.1 K | train
4 | model.text_embed | TokenEmbedding | 50.7 K | tr

In [36]:
# Mount Google Drive
import shutil
from google.colab import drive
drive.mount('/content/drive')

# Zip the outputs folder
shutil.make_archive('outputs', 'zip', 'outputs')

# Copy zipped file to Google Drive
shutil.copy('outputs.zip', '/content/drive/MyDrive/outputs_test.zip')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/outputs_test.zip'

In [9]:
import os

# Create a folder to unzip into
os.makedirs("outputs", exist_ok=True)

# Download
!wget -O outputs.zip https://huggingface.co/datasets/mirzasyhm/checkpoint_jawi/resolve/main/outputs_color.zip

# Unzip into that folder
!unzip -q outputs.zip -d outputs

# Clean up
!rm outputs.zip


--2025-06-08 14:49:15--  https://huggingface.co/datasets/mirzasyhm/checkpoint_jawi/resolve/main/outputs_color.zip
Resolving huggingface.co (huggingface.co)... 65.8.243.90, 65.8.243.46, 65.8.243.92, ...
Connecting to huggingface.co (huggingface.co)|65.8.243.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/a4/0e/a40edb5fee9aa56b081d76cebeb85749da8b1935a5194f415623e5924b1f8594/51ed8f3c2622c67b6b1135afb9b73ce7c65727cf6f70bd9ef1a6ddf9bebf208c?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27outputs_color.zip%3B+filename%3D%22outputs_color.zip%22%3B&response-content-type=application%2Fzip&Expires=1749396862&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0OTM5Njg2Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2E0LzBlL2E0MGVkYjVmZWU5YWE1NmIwODFkNzZjZWJlYjg1NzQ5ZGE4YjE5MzVhNTE5NGY0MTU2MjNlNTkyNGIxZjg1OTQvNTFlZDhmM2MyNjIyYzY3YjZiMTEzNWFmYjliNzNjZTdjNjU3MjdjZ

In [33]:
!ls outputs/parseq/2025-05-12_10-45-42/checkpoints

'epoch=32-step=12332-val_accuracy=97.8514-val_NED=99.0109.ckpt'
'epoch=33-step=12517-val_accuracy=97.9758-val_NED=99.0230.ckpt'
'epoch=34-step=12702-val_accuracy=98.1794-val_NED=99.0625.ckpt'
'epoch=39-step=13627-val_accuracy=98.5525-val_NED=99.1494.ckpt'
'epoch=43-step=14367-val_accuracy=98.6317-val_NED=99.1658.ckpt'
'epoch=44-step=14552-val_accuracy=98.5638-val_NED=99.1456.ckpt'
 last.ckpt
 last.ckpt.log.txt
 last-v1.ckpt
 last-v2.ckpt


In [38]:
!python test.py outputs/parseq/2025-05-12_10-45-42/checkpoints/last-v1.ckpt

Additional keyword arguments: {'charset_test': ' 0123456789۰۱۲٢۳۴۵۶۷۸۹اآأؤإءئۓۂئےۍېىيےیبپڀتٹثٿجچحخدڈذڎرڑزژسشصضطظعغفقڤڠݢکكڭگڬلمنںوۏههةۃۀہھڽضئکڤݢۏ-\u200c!"#$%&\'()*,./;<=>?@[\\]^_`{|}~۔'}
/usr/local/lib/python3.11/dist-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
jawi: 100% 18/18 [00:06<00:00,  2.61it/s]
Custom set:
| Dataset  | # samples | Accuracy | 1 - NED | Confidence | Label Length |
|:--------:|----------:|---------:|--------:|-----------:|-------------:|
| jawi     |      8843 |    98.22 |   99.14 |      98.38 |         6.41 |
|----------|-----------|----------|---------|------------|--------------|
| Combined |      8843 |    98.22 |   99.14 |      98.38 |         6.41 |




In [37]:
!python test_additional_metric.py \
        --checkpoint outputs/parseq/2025-05-12_10-45-42/checkpoints/last-v1.ckpt \
        --data_root data \
        --batch_size 32 \
        --num_workers 1 \
        --device cuda

Using charset (129 chars)
/usr/local/lib/python3.11/dist-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)

--- First 30 GT vs PR ---
01: GT='اخلاسکن' | PR='اخلاسکن'
02: GT='جيباکو' | PR='جيباکو'
03: GT='سمندا' | PR='سمندا'
04: GT='فوتوݢرف' | PR='فوتوݢرف'
05: GT='سندوڠن' | PR='سندوڠن'
06: GT='دڤادنکن۔' | PR='دڤادنکن۔'
07: GT='عفة' | PR='عفة'
08: GT='مڽومبوڠکن' | PR='مڽومبوڠکن'
09: GT='ݢماک' | PR='ݢماک'
10: GT='کيوي' | PR='کيوي'
11: GT='برمينيت۔' | PR='برمينيت۔'
12: GT='ڠيڠيق' | PR='ڠيڠيق'
13: GT='ݢاءي' | PR='ݢاءي'
14: GT='دسوسوءي' | PR='دسوسوءي'
15: GT='دأتس' | PR='دأتس'
16: GT='ريتينا۔' | PR='ريتينا۔'
17: GT='کدمڤيڠن' | PR='کدمڤيڠن'
18: GT='مربس' | PR='مربس'
19: GT='ڤمباچاءنڽ' | PR='ڤمباچاءنڽ'
20: GT='بابور' | PR='بابور'
21: GT='سکروم' | PR='سکروم'
22: GT='اوڤشن' | PR='اوڤشن'
23: GT='ترڤانچرله' | PR='ت

In [ ]:
import lmdb

LMDB_PATH = "data/test/jawi"  # ← adjust this to your split path
env = lmdb.open(LMDB_PATH, readonly=True, lock=False)
with env.begin() as txn:
    print("--- First 30 raw LMDB labels ---")
    for i in range(1, 31):
        key = f"label-{i:09d}".encode()
        val = txn.get(key)
        s = val.decode("utf-8", errors="replace") if val is not None else None
        print(f"{i:02d}: {repr(s)}")


--- First 30 raw LMDB labels ---
01: 'ڤاتوق'
02: 'دڤاتوق'
03: 'بيتالمال'
04: 'ڤتوڠوت'
05: 'مماتوتکن'
06: 'ڤاو'
07: 'دڤاءوق'
08: 'ڤاءوس'
09: 'ڤاءوس'
10: 'ڤاءوت'
11: 'برڤاءوت'
12: 'برباج'
13: 'ڤماءوت'
14: 'ڤرڤاءوتن'
15: 'ترڤاءوت'
16: 'برباج'
17: 'ڤاوه'
18: 'برڤايا-ڤايا'
19: 'ڤايه٢'
20: 'ڤاير٢'
21: 'دباجاءي'
22: 'برڤايو'
23: 'ممايوکن'
24: 'ڤايوق'
25: 'ڤايوڠن'
26: 'ڤير'
27: 'ڤچاچل'
28: 'برڤچه'
29: 'دڤچهکن'
30: 'ممچه'


In [24]:
!git pull


remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 409 bytes | 409.00 KiB/s, done.
From https://github.com/mirzasyhm/jawiOCR
   b84aacf..54a8bb7  main       -> origin/main
Updating b84aacf..54a8bb7
Fast-forward
 parseq_jawi/configs/main.yaml | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [ ]:
!python read.py outputs/parseq/2025-05-12_10-45-42/checkpoints/last.ckpt --images demo_images/*

Additional keyword arguments: {}
/usr/local/lib/python3.11/dist-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
demo_images/Image160967.jpg: ڤروجودن
demo_images/Image161005.jpg: يأجوجومأجوج
demo_images/Image161078.jpg: يتيم
demo_images/Image161571.jpg: زوفيت
demo_images/Image161653.jpg: کلابقکن
demo_images/Image168083.jpg: ملنتيڠ-لنتيڠ
demo_images/Image168799.jpg: برلتوق-لتوق
demo_images/Image182474.jpg: چوکايڽ
demo_images/Image183203.jpg: دڤرݢوناکنڽ


In [ ]:
from strhub.data.dataset import LmdbDataset

charset_test = ("0123456789۰۱۲٢۳۴۵۶۷۸۹اآأؤإءئۓۂئےۍېىيےیبپڀتٹثٿجچحخدڈذڎرڑزژسشصضطظعغفقڤڠݢکكڭگڬلمنںوۏههةۃۀہھڽضئکڤݢۏ-‌!\"#$%&'()*+,./:;<=>?@[\\]^_`{|}~")

ds = LmdbDataset(
    root="data/test/jawi",
    charset=charset_test,
    max_label_len=25         # same as model.max_label_length
)

print("Dataset length seen by LmdbDataset:", len(ds))


Dataset length seen by LmdbDataset: 19897


In [ ]:
import lmdb
import unicodedata
from collections import Counter
from strhub.data.utils import CharsetAdapter

# Configuration
lmdb_path = "data/test/jawi"
charset = ("0123456789۰۱۲٢۳۴۵۶۷۸۹اآأؤإءئۓۂئےۍېىيےیبپڀتٹثٿجچحخدڈذڎرڑزژسشصضطظعغفقڤڠݢکكڭگڬلمنںوۏههةۃۀہھڽضئکڤݢۏ-‌!\"#$%&'()*+,./:;<=>?@[\\]^_`{|}~"
)
max_label_len = 25

# Prepare adapter and counters
adapter = CharsetAdapter(charset)
dropped_long = []
dropped_unknown = []
unknown_chars = Counter()

# Open LMDB
env = lmdb.open(lmdb_path, readonly=True, lock=False)
with env.begin() as txn:
    total = int(txn.get(b'num-samples'))
    for idx in range(1, total + 1):
        # Fetch raw label
        key = f"label-{idx:09d}".encode()
        raw = txn.get(key)
        if raw is None:
            continue
        label = raw.decode('utf-8')
        # Preprocessing
        label_no_ws = ''.join(label.split())
        label_norm = unicodedata.normalize('NFKD', label_no_ws).encode('ascii', 'ignore').decode()
        # Check length
        if len(label_norm) > max_label_len:
            dropped_long.append((idx, label_norm))
            continue
        # Adapter mapping
        adapted = adapter(label_norm)
        if not adapted:
            # Track unknown characters
            for ch in set(label_norm):
                if ch not in charset:
                    unknown_chars[ch] += 1
         #   dropped_unknown.append((idx, label_norm))
        # Else it's kept

# Display summary
print(f"Total samples in LMDB      : {total}")
print(f"Dropped (too long)         : {len(dropped_long)}")
print(f"Dropped (unsupported chars): {len(dropped_unknown)}")
print("\nMost common unknown chars:")
for ch, cnt in unknown_chars.most_common(10):
    print(f"  {repr(ch)} (U+{ord(ch):04X}) x{cnt}")

print("\nFirst 5 too-long labels:")
for idx, lbl in dropped_long[:5]:
    print(f"  {idx:09d}: {lbl}")

print("\nFirst 5 labels with unknown chars:")
for idx, lbl in dropped_unknown[:5]:
    print(f"  {idx:09d}: {lbl}")


Total samples in LMDB      : 19897
Dropped (too long)         : 0
Dropped (unsupported chars): 0

Most common unknown chars:

First 5 too-long labels:

First 5 labels with unknown chars:


In [ ]:
from strhub.data.dataset import LmdbDataset

# Your Jawi charset string
charset = ("0123456789۰۱۲٢۳۴۵۶۷۸۹اآأؤإءئۓۂئےۍېىيےیبپڀتٹثٿجچحخدڈذڎرڑزژسشصضطظعغفقڤڠݢکكڭگڬلمنںوۏههةۃۀہھڽضئکڤݢۏ-‌!\"#$%&'()*+,./:;<=>?@[\\]^_`{|}~")

# Path to your LMDB
lmdb_path = "data/test/jawi"

# Instantiate the dataset (25 is your max_label_length)
ds = LmdbDataset(lmdb_path, charset, max_label_len=25)

print(f"Dataset length: {len(ds)} samples\n")
print("First 5 labels stored in the filtered dataset:")
for i in range(min(len(ds), 5)):
    _, label = ds[i]
    print(f"{i+1:2d}: {label}")


Dataset length: 19897 samples

First 5 labels stored in the filtered dataset:
 1: ڤاتوق
 2: دڤاتوق
 3: بيتالمال
 4: ڤتوڠوت
 5: مماتوتکن


In [ ]:
import lmdb

env = lmdb.open("data/test/jawi", readonly=True, lock=False)
with env.begin() as txn:
    count = 0
    cursor = txn.cursor()
    for key, _ in cursor:
        if not key.startswith(b'label-'):  # only count image keys
            continue
        count += 1

print("Image-label pairs in LMDB:", count)


Image-label pairs in LMDB: 19124
